In [1]:
from datasets import load_dataset
import openai
import os
import time
import pandas as pd
from scipy.spatial.distance import cosine
import pickle
from tqdm import tqdm 

/opt/anaconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset
dataset = load_dataset("sentence-transformers/stsb", split="train")
print(f"Loaded {len(dataset)} examples. Sample:")
print(dataset[0], "\n")


Loaded 5749 examples. Sample:
{'sentence1': 'A plane is taking off.', 'sentence2': 'An air plane is taking off.', 'score': 1.0} 



In [3]:
# Set your OpenAI API key (best via environment variable)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# Function to get embedding
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        model=model,
        input=[text]
    )
    # Extract the embedding vector
    return response.data[0].embedding

In [5]:
# Generate embeddings for the first few sentence pairs (do not run without need, save results, costs money)
file_to_save = 'openai_text-embedding-3-small.pickle'
embeddings = pd.DataFrame()
sentence1 = []
sentence2 = []
embedding1 = []
embedding2 = []
cosine_similarity = []
human_score = []

print('Extracting embedding and measuring similarity ...')
time.sleep(1)
for idx in tqdm(range(len(dataset))):
    first_sentence = dataset[idx]["sentence1"]
    emb1 = get_embedding(first_sentence)
    sentence1.append(
        first_sentence
    )
    embedding1.append(
        emb1
    )
    #############################
    second_sentence = dataset[idx]["sentence2"]
    emb2 = get_embedding(second_sentence)
    sentence2.append(
        second_sentence
    )
    embedding2.append(
        emb2
    )
    
    ############################
    cosine_similarity.append(
        1 - cosine(emb1, emb2)
    )
    human_score.append(
        dataset[idx]["score"]
    )
    
    time.sleep(1)  # respect rate limits!

embeddings['Sentence 1'] = sentence1
embeddings['Sentence 2'] = sentence2
embeddings['Embedding of Sentence 1'] = embedding1
embeddings['Embedding of Sentence 2'] = embedding2
embeddings['Cosine similarity'] = cosine_similarity
embeddings['Human score'] = human_score

with open(file_to_save, 'wb') as f:
    pickle.dump(embeddings, f)

Extracting embedding and measuring similarity ...


100%|██████████| 5749/5749 [2:40:54<00:00,  1.68s/it]  


In [6]:
embeddings.head()

,Sentence 1,Sentence 2,Embedding of Sentence 1,Embedding of Sentence 2,Cosine similarity,Human score
0,A plane is taking off.,An air plane is taking off.,"[-0.013315110467374325, 0.043752722442150116, ...","[-0.009698853828012943, 0.02614845149219036, 0...",0.922164,1.00
1,A man is playing a large flute.,A man is playing a flute.,"[0.029255172237753868, 0.002963797189295292, 0...","[-0.018380073830485344, 0.0007290428038686514,...",0.833169,0.76
2,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...,"[0.008580912835896015, -0.015802349895238876, ...","[0.020928939804434776, -0.014739884063601494, ...",0.927993,0.76
3,Three men are playing chess.,Two men are playing chess.,"[0.013227542862296104, 0.030965756624937057, 0...","[0.005606312304735184, 0.01778590679168701, 0....",0.857325,0.52
4,A man is playing the cello.,A man seated is playing the cello.,"[-0.004435026086866856, 0.004739934112876654, ...","[-0.0126184755936265, -0.01696082018315792, 0....",0.822483,0.85
